In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [2]:
df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_All_Transformed.csv")
#df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Transformed.csv")

In [3]:
rng=42

In [4]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [5]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [6]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [7]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home                                 1.000000
seasonPointsPerGame_Home                         0.301171
fenwickPercentageAvg_Home                        0.158833
xGoalsPercentageAvg_Home                         0.155926
scoreAdjustedUnblockedShotAttemptsForAvg_Home    0.155789
                                                   ...   
scoreAdjustedUnblockedShotAttemptsForAvg_Away   -0.146533
corsiPercentageAvg_Away                         -0.162194
fenwickPercentageAvg_Away                       -0.164881
xGoalsPercentageAvg_Away                        -0.166363
seasonPointsPerGame_Away                        -0.292426
Name: win_or_lose_Home, Length: 203, dtype: float64

In [8]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [9]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [10]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [11]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [12]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [13]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [14]:
# # baseline model
# from tensorflow import keras


# # create model
# base_model = keras.models.Sequential([
#     keras.Input(shape=(X_train.shape[1],)),
#     keras.layers.Dense(30, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
#                        ])
    
# # Compile model
# base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
def build_model(n_hidden=1, n_neurons=50, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        # Dropout layer will help with overfitting
        model.add(keras.layers.Dropout(rate=0.2))
        model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    return model

In [16]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2],
    'model__n_neurons': np.arange(1,30),
    'model__learning_rate' : reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, cv=3)

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=200,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

Epoch 1/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.7756 - val_loss: 0.6280
Epoch 2/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6302 - val_loss: 0.6153
Epoch 3/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6247 - val_loss: 0.6236
Epoch 4/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6166 - val_loss: 0.5918
Epoch 5/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5990 - val_loss: 0.5854
Epoch 6/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6047 - val_loss: 0.5870
Epoch 7/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5770 - val_loss: 0.5901
Epoch 8/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5935 - val_loss: 0.5765
Epoch 9/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5874 - val_loss: 0.5750
Epoch 10/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5933 - val_loss: 0.5681
Epoch 11/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5758 - val_loss: 0.5645
Epoch 12/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4392 - val_loss: 0.4859
Epoch 70/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4307 - val_loss: 0.4850
Epoch 71/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4190 - val_loss: 0.5119
Epoch 72/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4358 - val_loss: 0.5075
Epoch 73/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4254 - val_loss: 0.4858
Epoch 74/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4237 - val_loss: 0.4915
Epoch 75/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4522 - val_loss: 0.4950
Epoch 76/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4565 - val_loss: 0.4866
Epoch 77/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4156 - val_loss: 0.4929
Epoch 78/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4487 - val_loss: 0.4989
Epoch 79/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4412 - val_loss: 0.4946
Epoch 80/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4381 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3930 - val_loss: 0.4699
Epoch 138/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4120 - val_loss: 0.4607
Epoch 139/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3621 - val_loss: 0.4574
Epoch 140/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3774 - val_loss: 0.4715
Epoch 141/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3794 - val_loss: 0.4582
Epoch 142/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3781 - val_loss: 0.4763
Epoch 143/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3858 - val_loss: 0.4479
Epoch 144/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3886 - val_loss: 0.5023
Epoch 145/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3779 - val_loss: 0.4472
Epoch 146/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3863 - val_loss: 0.4582
Epoch 147/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3950 - val_loss: 0.4622
Epoch 148/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - lo

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6171 - val_loss: 0.5910
Epoch 5/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6078 - val_loss: 0.5885
Epoch 6/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5988 - val_loss: 0.5865
Epoch 7/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6055 - val_loss: 0.5843
Epoch 8/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6024 - val_loss: 0.5811
Epoch 9/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5776 - val_loss: 0.6027
Epoch 10/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5840 - val_loss: 0.5768
Epoch 11/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5747 - val_loss: 0.5766
Epoch 12/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5798 - val_loss: 0.5703
Epoch 13/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5707 - val_loss: 0.5679
Epoch 14/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5836 - val_loss: 0.5672
Epoch 15/200
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5607 - val

In [17]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [18]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

In [19]:
model = rnd_search_cv.best_estimator_.model

In [20]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=model, epochs=100, batch_size=5, verbose=0, random_state = 42,
                           loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])



In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#kfold = StratifiedKFold(n_splits=5, shuffle=True)
#results = cross_val_score(keras_clf, X_train_prepared, Y_train, cv=kfold)

#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


In [24]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

In [25]:
df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_All_Predict.csv")
#df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Predict.csv")

In [26]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T